In [1]:
# Requisites
from gensim.models import TfidfModel
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
import numpy as np
from scipy import spatial
from typing import Tuple, List
from datasets import load_dataset
import tensorflow as tf
from gensim.models import fasttext
from textsim_class import TextSimilarity
import spacy
from gensim.models import KeyedVectors
from gensim.models import word2vec

c:\Users\Usuario\Documents\Universitat\4rt Quatri\PLH\Practica4\word-embeddings\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
dataset = load_dataset("projecte-aina/sts-ca", trust_remote_code=True)

In [7]:
from gensim.models import FastText
import gensim
# No funciona de moment
model = gensim.models.fasttext.FastTextKeyedVectors.load('./models/fasttext/cc_ca_300.bin')

In [ ]:
wv_model = word2vec.Word2Vec.load(f"./models/word2vec_100_5.model")
wv_model = wv_model.wv

In [4]:
model = spacy.load("ca_core_news_trf")

In [5]:
model("hola que tal")

hola que tal

In [38]:
a = TextSimilarity(model=wv_model, dataset=dataset, mode="tfidf", rob=False)

In [ ]:
model

In [ ]:
a.model("aa")

aa

In [ ]:
a._map_roberta( "test", cls=True)

array([-8.85079652e-02, -4.10490245e-01, -2.88881093e-01,  9.61695090e-02,
       -3.41971964e-01,  1.97528854e-01, -2.20762551e-01, -3.32124412e-01,
       -4.34855402e-01,  1.80775821e-01,  1.77461863e-01,  3.05804610e-01,
       -4.90469933e-01,  3.75180781e-01,  1.92570791e-01,  2.31047869e-02,
        1.67903498e-01,  1.87374622e-01, -2.27856651e-01, -1.02829367e-01,
        4.33997065e-01,  1.73626706e-01, -2.23315477e-01,  1.31420881e-01,
       -1.00994676e-01, -2.87102550e-01, -7.13203788e-01,  7.09998235e-02,
        5.20927191e-01, -5.50654978e-02, -2.26446986e-02, -4.94334921e-02,
        4.59008515e-01, -5.81318215e-02,  3.68080765e-01, -3.33301961e-01,
        2.04239875e-01, -2.70322889e-01, -4.42092448e-01, -1.97019875e-01,
        3.09498459e-01,  5.16384959e-01, -1.85113907e-01, -2.74726093e-01,
        1.63949579e-01, -5.91075540e-01,  4.23032910e-01,  4.34954703e-01,
       -1.94442242e-01, -3.51993740e-03, -3.31355959e-01,  7.28655905e-02,
        2.35310137e-01,  

In [39]:
a.define_model()

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8       │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_9       │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 100)       │     10,100 │ input_layer_8[0]… │
│                     │                   │            │ input_layer_9[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_4 (Lambda)   │ (None)            │          0 │ dense_4[0][0],    │
│                     │                   │            │ dense_4[1][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 10,100 (39.45 KB)

 Trainable params: 10,100 (39.45 KB)

 Non-trainable params: 0 (0.00 B)

None


In [40]:
a.train()

Epoch 1/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.7762 - val_loss: 3.7799
Epoch 2/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3.8755 - val_loss: 3.6215
Epoch 3/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3.6873 - val_loss: 3.4885
Epoch 4/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3.5308 - val_loss: 3.3777
Epoch 5/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.4023 - val_loss: 3.2862
Epoch 6/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.2910 - val_loss: 3.2104
Epoch 7/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3.1998 - val_loss: 3.1464
Epoch 8/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.1172 - val_loss: 3.0916
Epoch 9/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 835us/step - loss: 3.0468 - val_loss: 3.0445
Epoch 10/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 878us/step - loss: 2.9818 - val_loss: 3.0039
Epoch 11/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.9290 - val_loss: 2.9691
Epoch 12/64
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.8751 - va

In [41]:
from scipy.stats import pearsonr
def compute_pearson(x_, y_):
    # Obtener las predicciones del modelo para los datos de prueba. En este ejemplo vamos a utilizar el corpus de training.
    y_pred = a.exec_model.predict(x_)
    # Calcular la correlación de Pearson entre las predicciones y los datos de prueba
    correlation, _ = pearsonr(y_pred.flatten(), y_.flatten())
    return correlation
print(f"Correlación de Pearson (baseline-train): {compute_pearson(a.x_train, a.y_train)}")
print(f"Correlación de Pearson (baseline-validation): {compute_pearson(a.x_val, a.y_val)}")
print(f"Correlación de Pearson (baseline-test): {compute_pearson(a.x_test, a.y_test)}")



65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 520us/step
Correlación de Pearson (baseline-train): 0.621267447092493
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 415us/step
Correlación de Pearson (baseline-validation): 0.4755218421897734
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 576us/step
Correlación de Pearson (baseline-test): 0.5362609155093417


In [44]:
from transformers import pipeline, AutoTokenizer
from scipy.special import logit

model = 'projecte-aina/roberta-base-ca-v2-cased-sts'
tokenizer = AutoTokenizer.from_pretrained(model)
pipe = pipeline('text-classification', model=model, tokenizer=tokenizer)

def prepare(sentence_pairs):
    sentence_pairs_prep = []
    for s1, s2 in sentence_pairs:
        sentence_pairs_prep.append(f"{tokenizer.cls_token} {s1}{tokenizer.sep_token}{tokenizer.sep_token} {s2}{tokenizer.sep_token}")
    return sentence_pairs_prep

sentence_pairs = [("El llibre va caure per la finestra.", "El llibre va sortir volant."),
                  ("M'agrades.", "T'estimo."),
                  ("M'agrada el sol i la calor", "A la Garrotxa plou molt.")]

predictions = pipe(prepare(sentence_pairs), add_special_tokens=False)

# convert back to scores to the original 0 and 5 interval
for prediction in predictions:
    prediction['score'] = logit(prediction['score'])
print(predictions)


c:\Users\Usuario\Documents\Universitat\4rt Quatri\PLH\Practica4\word-embeddings\.venv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Usuario\.cache\huggingface\hub\models--projecte-aina--roberta-base-ca-v2-cased-sts. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


[{'label': 'SIMILARITY', 'score': 2.1183004307689126}, {'label': 'SIMILARITY', 'score': 2.179974932297432}, {'label': 'SIMILARITY', 'score': 0.9511617858568939}]


In [49]:
input_pairs = [(e["sentence1"], e["sentence2"]) for e in dataset["train"]]
y_train = [e["label"] for e in dataset["train"]]
input_pairs_val = [(e["sentence1"], e["sentence2"]) for e in dataset["validation"]]
y_val = [e["label"] for e in dataset["validation"]]
input_pairs_test = [(e["sentence1"], e["sentence2"]) for e in dataset["test"]]
y_test = [e["label"] for e in dataset["test"]]

In [52]:
def prepare(sentence_pairs):
    sentence_pairs_prep = []
    for s1, s2 in sentence_pairs:
        sentence_pairs_prep.append(f"{tokenizer.cls_token} {s1}{tokenizer.sep_token}{tokenizer.sep_token} {s2}{tokenizer.sep_token}")
    return sentence_pairs_prep



def compute_pearson(input, y):
    # Obtener las predicciones del modelo para los datos de prueba. En este ejemplo vamos a utilizar el corpus de training.
    predictions = pipe(prepare(input), add_special_tokens=False)

    for prediction in predictions:
        prediction['score'] = logit(prediction['score'])

    y_pred = np.array([pred['score']for pred in predictions])
    # Calcular la correlación de Pearson entre las predicciones y los datos de prueba
    correlation, _ = pearsonr(y_pred.flatten(), np.array(y).flatten())
    return correlation

print(f"Correlación de Pearson (baseline-test): {compute_pearson(input_pairs_test, y_test)}")

Correlación de Pearson (baseline-test): 0.781988582023819
